Neste notebook, será realizados testes de qualidade de dados utilizando a biblioteca "great expectactions".
O arquivo a ser validado esta no diretório raiz e com o nome "vendas.csv".

In [1]:
import findspark
findspark.init()

In [2]:
# Importantdo as dependências
import string
from datetime import datetime
import great_expectations as ge
from great_expectations.dataset import SparkDFDataset

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [3]:
spark = SparkSession.builder.master("local").appName('spark_context')\
     .getOrCreate()

22/07/11 09:13:27 WARN Utils: Your hostname, FALCONI-d25645 resolves to a loopback address: 127.0.1.1; using 192.168.65.49 instead (on interface eth0)
22/07/11 09:13:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/11 09:13:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Definindo Schema do dataframe
schema_vendas =  T.StructType ([\
      T.StructField("data", T.StringType(), True),\
      T.StructField("cod_loja", T.IntegerType(), True),\
      T.StructField("cod_produto", T.IntegerType(), True),\
      T.StructField("qtd_vendas", T.IntegerType(), True),\
      T.StructField("lucro", T.StringType(), True),\
      T.StructField("tipo_venda", T.StringType(), True),\
      T.StructField("desconto", T.DoubleType(), True)
])

In [5]:
# Realizando a importação do arquivo csv
df_vendas = spark.read.format("csv").options(header='True', delimiter=';').schema(schema_vendas).csv("vendas.csv")

In [6]:
# Criando o dataframe com dados brutos que serão validados
df_vendas_validacao = ge.dataset.SparkDFDataset(df_vendas)

In [7]:
# Checando se coluna de desconto está null
df_vendas_validacao.expect_column_values_to_not_be_null('desconto')

{
  "meta": {},
  "result": {
    "element_count": 4871,
    "unexpected_count": 1277,
    "unexpected_percent": 26.21638267296243,
    "unexpected_percent_total": 26.21638267296243,
    "partial_unexpected_list": [
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null
    ]
  },
  "success": false,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [9]:
# Checando se coluna de data esta vazia
df_vendas_validacao.expect_column_values_to_not_be_null('data')

{
  "meta": {},
  "result": {
    "element_count": 4871,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [10]:
# Checando o formato da data
df_vendas_validacao.expect_column_values_to_match_strftime_format('data', "%Y-%m-%d", result_format={'result_format': 'SUMMARY'})

{
  "meta": {},
  "result": {
    "element_count": 4871,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 4871,
    "unexpected_percent": 100.0,
    "unexpected_percent_total": 100.0,
    "unexpected_percent_nonmissing": 100.0,
    "partial_unexpected_list": [
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019",
      "02/11/2019"
    ],
    "partial_unexpected_index_list": null,
    "partial_unexpected_counts": [
      {
        "value": "02/11/2019",
        "count": 20
      }
    ]
  },
  "success": false,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [11]:
# Exibindo o schema, 
# verificar qual linha
df_vendas_validacao.head()

,data,cod_loja,cod_produto,qtd_vendas,lucro,tipo_venda,desconto,__eval_col_desconto,__eval_col_data
0,02/11/2019,-28,32317,1,"13,49",DIRETO P.D.V,0.0,0.0,02/11/2019
1,02/11/2019,-28,16325,1,"7,98",ORCAMENTO,NaN,NaN,02/11/2019
2,02/11/2019,-18,21271,1,"15,82",ORCAMENTO,NaN,NaN,02/11/2019
3,02/11/2019,9,8005,1,"59,16",DIRETO P.D.V,0.0,0.0,02/11/2019
4,02/11/2019,21,28558,1,"24,12",ORCAMENTO,30.0,30.0,02/11/2019


In [12]:
type(df_vendas_validacao)

great_expectations.dataset.sparkdf_dataset.SparkDFDataset

In [13]:
# Checando se colunas são unicas (check de duplicação)
df_vendas_validacao.expect_column_values_to_be_unique("cod_produto")

{
  "meta": {},
  "result": {
    "element_count": 4871,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 3594,
    "unexpected_percent": 73.78361732703756,
    "unexpected_percent_total": 73.78361732703756,
    "unexpected_percent_nonmissing": 73.78361732703756,
    "partial_unexpected_list": [
      80,
      80,
      118,
      118,
      119,
      119,
      119,
      119,
      119,
      119,
      119,
      119,
      119,
      119,
      119,
      119,
      119,
      119,
      119,
      119
    ]
  },
  "success": false,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [14]:
# Checando se colunas possuem valor negativo
df_vendas_validacao.expect_column_values_to_be_between("cod_loja", min_value=0.0, result_format={'result_format': 'SUMMARY'})

{
  "meta": {},
  "result": {
    "element_count": 4871,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 3,
    "unexpected_percent": 0.06158899609936358,
    "unexpected_percent_total": 0.06158899609936358,
    "unexpected_percent_nonmissing": 0.06158899609936358,
    "partial_unexpected_list": [
      -28,
      -28,
      -18
    ],
    "partial_unexpected_index_list": null,
    "partial_unexpected_counts": [
      {
        "value": -28,
        "count": 2
      },
      {
        "value": -18,
        "count": 1
      }
    ]
  },
  "success": false,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [25]:
df_vendas_validacao.head()

,data,cod_loja,cod_produto,qtd_vendas,lucro,tipo_venda,desconto,__eval_col_desconto,__eval_col_data,__eval_col_cod_produto,__eval_col_cod_loja,__eval_col_tipo_venda
0,02/11/2019,-28,32317,1,"13,49",DIRETO P.D.V,0.0,0.0,02/11/2019,32317,-28,DIRETO P.D.V
1,02/11/2019,-28,16325,1,"7,98",ORCAMENTO,NaN,NaN,02/11/2019,16325,-28,ORCAMENTO
2,02/11/2019,-18,21271,1,"15,82",ORCAMENTO,NaN,NaN,02/11/2019,21271,-18,ORCAMENTO
3,02/11/2019,9,8005,1,"59,16",DIRETO P.D.V,0.0,0.0,02/11/2019,8005,9,DIRETO P.D.V
4,02/11/2019,21,28558,1,"24,12",ORCAMENTO,30.0,30.0,02/11/2019,28558,21,ORCAMENTO


In [15]:
# Check de valores negativos
df_vendas_validacao.expect_column_values_to_be_between("cod_loja", min_value=0.0, result_format={'result_format': 'SUMMARY'})

{
  "meta": {},
  "result": {
    "element_count": 4871,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 3,
    "unexpected_percent": 0.06158899609936358,
    "unexpected_percent_total": 0.06158899609936358,
    "unexpected_percent_nonmissing": 0.06158899609936358,
    "partial_unexpected_list": [
      -28,
      -28,
      -18
    ],
    "partial_unexpected_index_list": null,
    "partial_unexpected_counts": [
      {
        "value": -28,
        "count": 2
      },
      {
        "value": -18,
        "count": 1
      }
    ]
  },
  "success": false,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [16]:
df_vendas_validacao.expect_column_values_to_be_between("cod_loja", min_value=0.0, result_format={'result_format': 'BOOLEAN_ONLY'})

{
  "meta": {},
  "result": {},
  "success": false,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [16]:
# Checando se os campos estão com o tipo correto
df_vendas_validacao.expect_column_values_to_be_of_type("tipo_venda", "DoubleType", result_format={'result_format': 'SUMMARY'})

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": "StringType"
  },
  "success": false
}

In [17]:
# Checando se os campos estão com o tipo correto
df_vendas_validacao.expect_column_values_to_be_of_type("tipo_venda", "StringType", result_format={'result_format': 'BASIC'})

{
  "meta": {},
  "result": {
    "observed_value": "StringType"
  },
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [19]:
df_vendas_validacao.expect_column_values_to_be_of_type("desconto", "DoubleType")

{
  "meta": {},
  "result": {
    "observed_value": "DoubleType"
  },
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [24]:
# Checando ordem das colunas
df_vendas_validacao.expect_table_columns_to_match_ordered_list(column_list=['data', 'cod_loja', 'cod_produto', 'qtd_vendas', 'lucro', 'tipo_venda', 'desconto', '__eval_col_desconto', '__eval_col_data', '__eval_col_cod_produto', '__eval_col_cod_loja', '__eval_col_tipo_venda'])

{
  "meta": {},
  "result": {
    "observed_value": [
      "data",
      "cod_loja",
      "cod_produto",
      "qtd_vendas",
      "lucro",
      "tipo_venda",
      "desconto",
      "__eval_col_desconto",
      "__eval_col_data",
      "__eval_col_cod_produto",
      "__eval_col_cod_loja",
      "__eval_col_tipo_venda"
    ]
  },
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}